# Dynet segmentation with tf fold

In [20]:
#just a bunch of fun
import numpy as np
import six
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td

# params
symbol='0123456789-,;.!?:’\”\"/\\|_@#$%^&*~`+ =<>()[]{}'
accent = 'áéíóöőúüű'
alphabet = 'abcdefghijklmnopqrstuvwxyz'
end_of_sentence = "."
SPLIT_CHAR = '\t'

#our alphabet
hunabc = ' ' + alphabet + accent
hunabc += hunabc.upper()
hunabc += end_of_sentence
vocabulary = sorted(set(hunabc))
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]

## helper functions

In [23]:
def char_split(input_line, delimiter=''):
    source, target = tf.string_split(input_line, delimiter=delimiter)
    return source, target

def decode(string):
    string=str(string)
    return string.decode('utf-8')

def read_line(filename_queue):
    global SPLIT_CHAR
    reader = tf.TextLineReader(skip_header_lines=0)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[""], [""]]
    source, target = tf.decode_csv(csv_row, record_defaults=record_defaults, field_delim=SPLIT_CHAR)
    
    return {"in": source,"out": target}


In [3]:
filenames=["data.txt"]
filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
data = read_line(filename_queue)


sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

while not coord.should_stop():
    try:
        s= sess.run(data)
        print(s["in"].decode("utf-8"))
    except tf.errors.OutOfRangeError:
        break
coord.request_stop()
coord.join(threads)

áááááááááááá
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
harom valami cucc
egy valami cucc
h

In [55]:

oneblock = td.OneHot(len(vocabulary), dtype='float32', name="onehot_encoder")
length = td.Length()


demo_word=["a","e","k"]
inputs =  td.InputTransform(lambda s: [index(x) for x in s]) >> td.Map(td.Scalar('int32')) #>> td.Function(td.Embedding(128, 8))
print(inputs.eval(" abcdef"))

[array(0, dtype=int32), array(28, dtype=int32), array(29, dtype=int32), array(30, dtype=int32), array(31, dtype=int32), array(32, dtype=int32), array(33, dtype=int32)]


In [5]:
def conv1d_on_variable_sequence(x, scope, kernel_size=3, input_channels=71, output_channels=71):
    with tf.variable_scope(scope) as sc:
        filter = tf.get_variable("conv_filter", [kernel_size] +  [input_channels, output_channels] , initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable("conv_bias",  output_channels, initializer=tf.constant_initializer(0.05, dtype=tf.float32))
        conv = tf.nn.conv1d(x, filters=filter, stride=1, padding='SAME')
        return tf.nn.relu(tf.add(conv, bias))
    

In [18]:
vec = td.Tensor([len(vocabulary)])
a = td.Map(vec) >> td.NGrams(3)
a.eval([oneblock.eval("k"),oneblock.eval("a"),oneblock.eval("c"),oneblock.eval("s"),oneblock.eval("a")])
b = td.Zeros([3,71]) >> td.ScopedLayer(conv1d_on_variable_sequence)
c = a >> td.GetItem(0)

In [17]:
b.eval([oneblock.eval("k"),oneblock.eval("a"),oneblock.eval("c"),oneblock.eval("s"),oneblock.eval("a")])

array([[ 0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05],
       [ 0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,  0.05,
         0.0